In [140]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator

In [141]:
# Load data
y = pd.read_csv("input/Ospiti.csv")
y["Data"] = pd.to_datetime(y["Data"]).dt.date
y 

,Data,Ospiti
0,2021-06-01,319.0
1,2021-06-02,396.0
2,2021-06-03,188.0
3,2021-06-04,258.0
4,2021-06-05,571.0
...,...,...
542,2022-11-26,732.0
543,2022-11-27,495.0
544,2022-11-28,226.0
545,2022-11-29,289.0


In [142]:
# dates to be predicted
dates = pd.date_range(start=y["Data"].iloc[-1],periods=8)[1:]
dates

DatetimeIndex(['2022-12-01', '2022-12-02', '2022-12-03', '2022-12-04',
               '2022-12-05', '2022-12-06', '2022-12-07'],
              dtype='datetime64[ns]', freq='D')

In [143]:
# normalize data with MinMaxScaler
scaler = MinMaxScaler()
y_scaled = scaler.fit_transform(y["Ospiti"].values.reshape(-1,1))
y_scaled = np.reshape(y_scaled, (1, len(y_scaled)))
y_scaled

array([[0.18949343, 0.26172608, 0.06660413, 0.13227017, 0.42589118,
        0.34803002, 0.09099437, 0.13320826, 0.24015009, 0.21294559,
        0.20262664, 0.33395872, 0.22045028, 0.09662289, 0.20731707,
        0.0750469 , 0.14446529, 0.21857411, 0.35928705, 0.21388368,
        0.09380863, 0.14446529, 0.20356473, 0.2054409 , 0.22045028,
        0.14821764, 0.19512195, 0.15572233, 0.18761726, 0.1641651 ,
        0.1575985 , 0.09474672, 0.34990619, 0.29268293, 0.17354597,
        0.07879925, 0.21763602, 0.26547842, 0.22326454, 0.33020638,
        0.06378987, 0.0891182 , 0.17260788, 0.16979362, 0.21669794,
        0.18667917, 0.43621013, 0.17729831, 0.19699812, 0.16510319,
        0.20450281, 0.20168856, 0.17917448, 0.33020638, 0.29831144,
        0.15478424, 0.17448405, 0.21482176, 0.18105066, 0.22795497,
        0.39118199, 0.24859287, 0.22326454, 0.18761726, 0.28611632,
        0.27861163, 0.16510319, 0.22514071, 0.14540338, 0.12382739,
        0.14071295, 0.12007505, 0.2532833 , 0.16

In [144]:
tf_dataset = tf.data.Dataset.from_tensor_slices(y_scaled[0])

In [145]:
def make_window_dataset(ds, window_size=5, shift=1, stride=1):
  windows = ds.window(window_size, shift=shift, stride=stride)

  def sub_to_batch(sub):
    return sub.batch(window_size, drop_remainder=True)

  windows = windows.flat_map(sub_to_batch)
  return windows

In [146]:
ds = make_window_dataset(tf_dataset, window_size=30, shift = 1, stride=1)

for example in ds.take(10):
  print(example.numpy())

[0.18949343 0.26172608 0.06660413 0.13227017 0.42589118 0.34803002
 0.09099437 0.13320826 0.24015009 0.21294559 0.20262664 0.33395872
 0.22045028 0.09662289 0.20731707 0.0750469  0.14446529 0.21857411
 0.35928705 0.21388368 0.09380863 0.14446529 0.20356473 0.2054409
 0.22045028 0.14821764 0.19512195 0.15572233 0.18761726 0.1641651 ]
[0.26172608 0.06660413 0.13227017 0.42589118 0.34803002 0.09099437
 0.13320826 0.24015009 0.21294559 0.20262664 0.33395872 0.22045028
 0.09662289 0.20731707 0.0750469  0.14446529 0.21857411 0.35928705
 0.21388368 0.09380863 0.14446529 0.20356473 0.2054409  0.22045028
 0.14821764 0.19512195 0.15572233 0.18761726 0.1641651  0.1575985 ]
[0.06660413 0.13227017 0.42589118 0.34803002 0.09099437 0.13320826
 0.24015009 0.21294559 0.20262664 0.33395872 0.22045028 0.09662289
 0.20731707 0.0750469  0.14446529 0.21857411 0.35928705 0.21388368
 0.09380863 0.14446529 0.20356473 0.2054409  0.22045028 0.14821764
 0.19512195 0.15572233 0.18761726 0.1641651  0.1575985  0.094

In [147]:
def dense_7_step(batch):
  # Shift features and labels one step relative to each other.
  return batch[:-7], batch[-7:]

In [ ]:
# create tensor from a list of tensors

In [170]:
dense_labels_ds = ds.map(dense_7_step)

#create a tensor for holding the labels and features
all_inputs = []
all_labels = []

for inputs,labels in dense_labels_ds.take(3):
  print(inputs.numpy(), "=>", labels.numpy())

for inputs,labels in dense_labels_ds:
  all_inputs.append(inputs)
  all_labels.append(labels)

[0.18949343 0.26172608 0.06660413 0.13227017 0.42589118 0.34803002
 0.09099437 0.13320826 0.24015009 0.21294559 0.20262664 0.33395872
 0.22045028 0.09662289 0.20731707 0.0750469  0.14446529 0.21857411
 0.35928705 0.21388368 0.09380863 0.14446529 0.20356473] => [0.2054409  0.22045028 0.14821764 0.19512195 0.15572233 0.18761726
 0.1641651 ]
[0.26172608 0.06660413 0.13227017 0.42589118 0.34803002 0.09099437
 0.13320826 0.24015009 0.21294559 0.20262664 0.33395872 0.22045028
 0.09662289 0.20731707 0.0750469  0.14446529 0.21857411 0.35928705
 0.21388368 0.09380863 0.14446529 0.20356473 0.2054409 ] => [0.22045028 0.14821764 0.19512195 0.15572233 0.18761726 0.1641651
 0.1575985 ]
[0.06660413 0.13227017 0.42589118 0.34803002 0.09099437 0.13320826
 0.24015009 0.21294559 0.20262664 0.33395872 0.22045028 0.09662289
 0.20731707 0.0750469  0.14446529 0.21857411 0.35928705 0.21388368
 0.09380863 0.14446529 0.20356473 0.2054409  0.22045028] => [0.14821764 0.19512195 0.15572233 0.18761726 0.1641651  0.

(TensorSpec(shape=(23,), dtype=tf.float64, name=None),
 TensorSpec(shape=(7,), dtype=tf.float64, name=None))

In [172]:
prova = tf.concat(all_inputs, axis=0)
prova

<tf.Tensor: shape=(11914,), dtype=float64, numpy=
array([0.18949343, 0.26172608, 0.06660413, ..., 0.12382739, 0.19418386,
       0.18574109])>

### Build a simple dense model

In [174]:
# build a forecasting model using the Keras Sequential API
model = keras.Sequential(
    [
        layers.Dense(30, activation="relu", name="layer1"),
        layers.Dense(128, activation="relu", name="layer2"),
        layers.Dense(64, activation="relu", name="layer3"),
        layers.Dense(7, name="layer4"),
    ]
)

In [175]:
model.compile(optimizer='adam', loss="mse")

In [162]:
model.fit(x=all_inputs,y=all_labels,epochs=100)

Epoch 1/100
Consider rewriting this model with the Functional API.


ValueError: in user code:

    File "/home/davide/.local/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/home/davide/.local/lib/python3.8/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/davide/.local/lib/python3.8/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/home/davide/.local/lib/python3.8/site-packages/keras/engine/training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "/home/davide/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/davide/.local/lib/python3.8/site-packages/keras/engine/input_spec.py", line 199, in assert_input_compatibility
        raise ValueError(